In [65]:
#Importando as bibliotecas que vamos usar
import pandas as pd;
import numpy as np;
np.seterr(over='ignore')
import math;
from scipy.spatial import distance

In [66]:
# Dicionário para mapear os 22 atributos, a cada indíce. O primeiro atributo pode ser b,c,x,f,k,s a quem deve-se atribuir 0 para n, 1 para c, 2 para x, 3 para f, 4 para k e 5 para s. Fazemos o primeiro elemento de preparation ser preparation[1] = 'bcxfks' assim, podemos fazer a troca pela indexação da string. Ex: preparation[1].index('c') = 2, preparation[5].index('y') = 3
preparation = {1: 'bcxfks', 2: 'fgys', 3: 'nbcgrpuewy', 4: 'tf', 5: 'alcyfmnps', 6: 'adfn', 7: 'cwd', 8: 'bn',
               9: 'knbhgropuewy', 10: 'et', 11: 'bcuezr?', 12: 'fyks', 13: 'fyks', 14: 'nbcgopewy', 15: 'nbcgopewy',
               16: 'pu', 17: 'nowy', 18: 'not', 19: 'ceflnpsz', 20: 'knbhrouwy', 21: 'acnsvy', 22: 'glmpuwd'}

In [67]:
# Número de vizinhos
knn = int(input())
n = int(input())



3
6000


In [68]:
#Função para ler a entrada em CSV e retornar como matriz Numpy
def readcsv(filename):
    data = pd.read_csv(filename) 
    return(np.array(data)) 



In [69]:
#Arquivo de Entrada
data = readcsv('mushroom.csv')

In [70]:
# Número de linhas da matriz x_train
n_train = n;
# Numero de linhas da matriz x_test
n_test = len(data) - n;

#Leitura de x_train e y_train
x_train = data[0:n_train,0:22];
y_train = data[0:n_train,22:23];


In [71]:
# A função modeling recebe a matriz x_train e devolve a matriz df. A matriz df é o conjunto de dados númericos da matriz x_train e tem tamanho igual a matriz x_train.
def modeling(n_train, x_train):
    df = []
    # Esse laço percorrerá as k linhas da matriz x_train e criará uma lista temporária df_temp.
    for k in range(n_train):
        i = 1
        df_temp = []
        # Esse laço percorrerá cada coluna da linha fixada k. E cada valor numerico de preparation[i].index(x_train[k][i-1]) será adicionado a lista df_temp. Assim o k está fixo e o i variando de 1 até 22. Tem i-1 pois o primeiro valor de x_train é x_train[0][0] enquanto o dicionário começa por preparation[1]. 
        while i != 23:
            df_temp.append(preparation[i].index(x_train[k][i - 1]))
            i += 1
        # Por fim, com o fim do laço while, teremos df_temp sendo uma matriz com uma linha e 22 colunas dos valores númericos da linha fixada de x_train. Assim adicionamos df_temp a df. Df terá tamanho igual a x_train.
        df.append(df_temp)
    return df

In [72]:
# Salvamos a matriz de treinamento com os valores númericos como sendo df_x
df_x = modeling(n_train, x_train)

#Matriz Numpy
df_x = np.array(df_x);


In [73]:
# Calculamos a matriz transposta de df_x como transposed_X. 

transposed_x = np.transpose(df_x);

In [74]:
# Calculamos a matriz das medias da matriz tranposta como mean_x. 
mean_x = transposed_x.mean(axis=1)
tranposed_mean = np.transpose(mean_x);


In [75]:
# Calculamos a lista dos valores do desvio padrão de cada linha da matriz transposed_x. 
standard_deviation_x = transposed_x.std(axis=1)


In [76]:
#Salvamos como df_normalized_x, a matriz df_x normalizada. 

a = df_x - tranposed_mean
c = a/standard_deviation_x;

#Fazemos o tratamento para std = 0;
df_normalized_x = np.nan_to_num(c,copy=True, neginf=0, posinf=0)


<ipython-input-76-cf3c17fc1828>:4: RuntimeWarning: invalid value encountered in true_divide
  c = a/standard_deviation_x;


In [77]:
#Criar um dicionário para associar cada linha da matriz treinamento a um valor "p" ou "e" da lista y_train. Vai ficar: {1 : 'p', 2: 'e', ... , n_train: 'p'}
dic_df_x = {}
i = 0
for x in df_x:
    dic_df_x[i] = y_train[i]
    i += 1

In [78]:
#Aqui vamos ler os n_test valores da matriz teste.

aux = data[n_train:len(data),0:23];

x_test = aux[:,0:22];
y_test = data[:,22:23];

y_test = data[n_train:len(data),22:23];


In [79]:
#Aqui vamos transformá-la numa matriz númerica.
df_x_test = modeling(n_test, x_test)


#Matriz Numpy
df_x_test = np.array(df_x_test);


In [80]:
#Salvamos como df_normalized_x, a matriz df_x normalizada. 

a = df_x_test - tranposed_mean
c = a/standard_deviation_x;


#Fazemos o tratamento para std = 0;
df_x_test_normalized = np.nan_to_num(c,copy=True, neginf=0, posinf=0)
 

<ipython-input-80-0b2b2b14bd50>:4: RuntimeWarning: divide by zero encountered in true_divide
  c = a/standard_deviation_x;
<ipython-input-80-0b2b2b14bd50>:4: RuntimeWarning: invalid value encountered in true_divide
  c = a/standard_deviation_x;


In [81]:
#Aqui vamos calcular a distancia euclidiana de cada linha i da matriz normalizada de teste, com cada uma das linhas da matriz normalizada de treinamento.
distances_total = []
c = 0;
#Vamos iterar cada uma das i linhas da matriz normalizada de teste. Com um contador, k, e uma lista temporário distances2.
for i in df_x_test_normalized:
    
    #Acompanhar andamento da iteração - Se não quiser o print é só comentar a linha
    print("Iteração: ",c,"/",n_test);
    c = c+1;
    
    distances2 = []
    k = 0
    #Aqui, iterar cada k linha da matriz normalizada de treinamento. Criando uma segunda lista temporário, distances1 e um contador y.
    for k in df_normalized_x:
        
        #Mahalanobis Calculation - Slow Algorithm - + - 1h
        #V = np.cov(np.array([i, k]).T)
        #iv = np.linalg.pinv(V)
        #distances2.append(distance.mahalanobis(i, k, iv))
        
        #The Jensen-Shannon distance 
        distances2.append(distance.jensenshannon(i,k));

        #Euclidian Distance Algorithm
        #distances2.append(np.linalg.norm(i - k))
    #Por fim, adicionamos cada resultado de distances2, a matriz distances_total. A matriz distances_total terá n_test linhas e 22 colunas.
    distances_total.append(distances2)

Iteração:  0 / 2124
Iteração:  1 / 2124
Iteração:  2 / 2124
Iteração:  3 / 2124
Iteração:  4 / 2124
Iteração:  5 / 2124
Iteração:  6 / 2124
Iteração:  7 / 2124
Iteração:  8 / 2124
Iteração:  9 / 2124
Iteração:  10 / 2124
Iteração:  11 / 2124
Iteração:  12 / 2124
Iteração:  13 / 2124
Iteração:  14 / 2124
Iteração:  15 / 2124
Iteração:  16 / 2124
Iteração:  17 / 2124
Iteração:  18 / 2124
Iteração:  19 / 2124
Iteração:  20 / 2124
Iteração:  21 / 2124
Iteração:  22 / 2124
Iteração:  23 / 2124
Iteração:  24 / 2124
Iteração:  25 / 2124
Iteração:  26 / 2124
Iteração:  27 / 2124
Iteração:  28 / 2124
Iteração:  29 / 2124
Iteração:  30 / 2124
Iteração:  31 / 2124
Iteração:  32 / 2124
Iteração:  33 / 2124
Iteração:  34 / 2124
Iteração:  35 / 2124
Iteração:  36 / 2124
Iteração:  37 / 2124
Iteração:  38 / 2124
Iteração:  39 / 2124
Iteração:  40 / 2124
Iteração:  41 / 2124
Iteração:  42 / 2124
Iteração:  43 / 2124
Iteração:  44 / 2124
Iteração:  45 / 2124
Iteração:  46 / 2124
Iteração:  47 / 2124
It

In [82]:
#Aqui vamos associar as distancias ao dicionário dic_df_x para sabermos se o vizinho é comestível, 'p', ou venenoso, 'e'.     
a = []
#Vamos percorrer o tamanho de n_test, pois n_test é igual a quantidade de linhas da matriz distances_total. E criamos uma lista temporária.
for i in range(n_test):
    #Acompanhar andamento da iteração -  Se não quiser o print é só comentar a linha
    print("Iteração: ",i,"/",n_test);
    lista = []
    #Aqui vamos descobrir cada rótulo do j vizinho mais próximo. knn é o primeiro input do trabalho.
    for j in range(knn):
        #Vamos adicionar a lista 'lista' o rótulo dos vizinhos.
        #distances_total[i].index(sorted(distances_total[i])[j]): 
        #a lista sorted(distances_total[i]) é a lista distance_total[i] ordenada, 
        #.index(sorted(distances_total[i])[j]) retorna o indice do j-esimo elemento da linha i da matriz distances_total. Lembrando q 0 <=0 j <= knn. Assim j=0 é o vizinho mais proximo, j = knn é o knn-esimo vizinho mais proximo.
        #dic_df_x[distances_total[i].index(sorted(distances_total[i])[j])] >> retorna o rótulo 'p' ou 'e' da linha de teste i dos j vizinhos mais próximos. 
        #Lista terá uma linha e knn colunas.
        lista.append(dic_df_x[distances_total[i].index(sorted(distances_total[i])[j])])
    #a terá n_test linhas e knn colunas.
    a.append(lista)

Iteração:  0 / 2124
Iteração:  1 / 2124
Iteração:  2 / 2124
Iteração:  3 / 2124
Iteração:  4 / 2124
Iteração:  5 / 2124
Iteração:  6 / 2124
Iteração:  7 / 2124
Iteração:  8 / 2124
Iteração:  9 / 2124
Iteração:  10 / 2124
Iteração:  11 / 2124
Iteração:  12 / 2124
Iteração:  13 / 2124
Iteração:  14 / 2124
Iteração:  15 / 2124
Iteração:  16 / 2124
Iteração:  17 / 2124
Iteração:  18 / 2124
Iteração:  19 / 2124
Iteração:  20 / 2124
Iteração:  21 / 2124
Iteração:  22 / 2124
Iteração:  23 / 2124
Iteração:  24 / 2124
Iteração:  25 / 2124
Iteração:  26 / 2124
Iteração:  27 / 2124
Iteração:  28 / 2124
Iteração:  29 / 2124
Iteração:  30 / 2124
Iteração:  31 / 2124
Iteração:  32 / 2124
Iteração:  33 / 2124
Iteração:  34 / 2124
Iteração:  35 / 2124
Iteração:  36 / 2124
Iteração:  37 / 2124
Iteração:  38 / 2124
Iteração:  39 / 2124
Iteração:  40 / 2124
Iteração:  41 / 2124
Iteração:  42 / 2124
Iteração:  43 / 2124
Iteração:  44 / 2124
Iteração:  45 / 2124
Iteração:  46 / 2124
Iteração:  47 / 2124
It

In [83]:
b = [];

for i in range(n_test):
    
    #Aqui checamos se a maioria dos knn vizinhos mais proximos da linha i da matriz test são comestiveis, se sim imprimimos p, se não imprimimos e.
    if a[i].count('p') > (knn / 2):
        print('p')
        b.append('p');
    else:
        print('e')
        b.append('e');


p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p


In [84]:
#Comparar Y Classificado com Y Verdadeiro
acum = 0;
i = 0;
for i in range(n_test):
    if b[i] == y_test[i]:
        acum = acum + 1;
    

In [85]:
#Acurácia
print("Acurácia do Classificador Exemplo:",acum*100/n_test,"%")

Acurácia do Classificador Exemplo: 75.14124293785311 %
